# CMM202 Topic 8 Lab (Solved)

In [1]:
# install necessary packages
!pip install requests
!pip install bs4

In [2]:
# import required packages
import requests
from bs4 import BeautifulSoup

In [3]:
# define the url
url = "https://en.wikipedia.org/wiki/List_of_towns_and_cities_in_Scotland_by_population"
# request the url
r = requests.get(url)
# soup it
soup = BeautifulSoup(r.content,"html.parser")
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of towns and cities in Scotland by population - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"7c87ac90-3a5c-4a9f-9c07-e126740e9cd0","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_towns_and_cities_in_Scotland_by_population","wgTitle":"List of towns and cities in Scotland by population","wgCurRevisionId":1066348351,"wgRevisionId":1066348351,"wgArticleId":14184004,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short 

In [4]:
# get the rows of the table
rows = soup.find("table", {"class":"wikitable sortable"}).find_all("tr")
rows

[<tr>
 <th>Rank</th>
 <th>Locality</th>
 <th>Population</th>
 <th>Status</th>
 <th>Council area
 </th></tr>, <tr>
 <th>1
 </th>
 <td><a href="/wiki/Glasgow" title="Glasgow">Glasgow</a><span class="anchor" id="Glasgow"></span>
 </td>
 <td>612,040
 </td>
 <td>City
 </td>
 <td><a href="/wiki/Glasgow_City_Council" title="Glasgow City Council">Glasgow City</a>
 </td></tr>, <tr>
 <th>2
 </th>
 <td><a href="/wiki/Edinburgh" title="Edinburgh">Edinburgh</a><span class="anchor" id="Edinburgh"></span>
 </td>
 <td>488,050
 </td>
 <td>City
 </td>
 <td><a href="/wiki/City_of_Edinburgh_Council" title="City of Edinburgh Council">City of Edinburgh</a>
 </td></tr>, <tr>
 <th>3
 </th>
 <td><a href="/wiki/Aberdeen" title="Aberdeen">Aberdeen</a><span class="anchor" id="Aberdeen"></span>
 </td>
 <td>200,680
 </td>
 <td>City
 </td>
 <td><a href="/wiki/Aberdeen_City_Council" title="Aberdeen City Council">Aberdeen City</a>
 </td></tr>, <tr>
 <th>4
 </th>
 <td><a href="/wiki/Dundee" title="Dundee">Dundee</a><sp

In [5]:
# As an example, you can see one row
# 0 is the header, so there is th not td!
rows[0].find_all("th")

[<th>Rank</th>,
 <th>Locality</th>,
 <th>Population</th>,
 <th>Status</th>,
 <th>Council area
 </th>]

In [6]:
# This is an actual row!
rows[1].find_all("td")

[<td><a href="/wiki/Glasgow" title="Glasgow">Glasgow</a><span class="anchor" id="Glasgow"></span>
 </td>, <td>612,040
 </td>, <td>City
 </td>, <td><a href="/wiki/Glasgow_City_Council" title="Glasgow City Council">Glasgow City</a>
 </td>]

In [7]:
# create two lists to put names and populations
cities = []
population = []
# Go over al rows to find the cities 
# i.e entry with the word "city" in the third column
# "if cell" lets me skip the header
for row in rows:
    cells = row.find_all("td")
    if cells and "City" in cells[2].get_text(): 
        # if city, get name and pop
        cities.append(cells[0].get_text().split("\n")[0])
        population.append(int(cells[1].get_text().split("\\")[0].replace(",","")))
print(cities,population)

['Glasgow', 'Edinburgh', 'Aberdeen', 'Dundee', 'Perth', 'Inverness', 'Stirling'] [612040, 488050, 200680, 148280, 47430, 47290, 37610]


This is the data converted into a pandas data frame:

In [8]:
import pandas as pd
scotland = pd.DataFrame({'City':cities,'Population':population})
scotland

,City,Population
0,Glasgow,612040
1,Edinburgh,488050
2,Aberdeen,200680
3,Dundee,148280
4,Perth,47430
5,Inverness,47290
6,Stirling,37610


Another (more complicated option) is to scrap the data from each Wikipedia entry:
     
    https://en.wikipedia.org/wiki/Dundee
    https://en.wikipedia.org/wiki/Edinburgh
    https://en.wikipedia.org/wiki/Glasgow
    https://en.wikipedia.org/wiki/Aberdeen
    https://en.wikipedia.org/wiki/Perth,_Scotland
    https://en.wikipedia.org/wiki/Inverness
    https://en.wikipedia.org/wiki/Stirling

In [9]:
### To test the different websites, uncomment the url

url = "https://en.wikipedia.org/wiki/Dundee"
#url = "https://en.wikipedia.org/wiki/Edinburgh"
#url = "https://en.wikipedia.org/wiki/Glasgow"
#url = "https://en.wikipedia.org/wiki/Aberdeen"
#url = "https://en.wikipedia.org/wiki/Perth,_Scotland"
#url = "https://en.wikipedia.org/wiki/Inverness"
#url = "https://en.wikipedia.org/wiki/Stirling"

import requests
from bs4 import BeautifulSoup
pop=int()
r = requests.get(url)
soup = BeautifulSoup(r.content,"html.parser")

if "Stirling" in url:
    rows = soup.find("table", {"class":"infobox ib-uk-place vcard"}).find_all("tr")
else:
    rows = soup.find("table", {"class":"infobox ib-settlement vcard"}).find_all("tr")

for i, row in enumerate(rows):
    cells = row.find_all("th")
    if cells and "population" in cells[0].get_text().lower():
        if row.find_all("td"):
            data_cells = row.find_all("td")
        else:
            data_cells = rows[i+1].find_all("td") # The entry is AFTER the population!
        pop = int(data_cells[0].get_text().split("(")[0].split("[")[0].strip().replace(",",""))
print(pop)

148280


As you can see this solution is not very optimal, since you have to consider different pages, syntax, class names for tables, etc.